In [5]:
from gensim.models import KeyedVectors

In [6]:
word_vectors = KeyedVectors.load_word2vec_format('deps.words', binary=False) 

In [23]:
with open('deps.words', 'r') as f:
    text = f.read()
    length = len(text.split('\n'))-1
    dim = len(text.split('\n')[1].split())-1
    print(length, dim)
    
#text = '{} 300\n'.format(length) + text

#with open('deps.words', 'w+') as f:
  #  f.write(text)

174016 300


In [7]:
print(word_vectors)

In [33]:
word_vectors.similarity('technology', 'ruben')

0.2699586525433222